In [ ]:
import lib.display
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import animation
from skimage import transform
from IPython.display import HTML, clear_output
import ipywidgets as widgets

from math import floor
import string

First we load in all the events that have been randomly selected for calibration, as well as the 9x9 windows so that you can actually visualize those events. We also set the 'score' column to be empty. Rerunning this cell will erase your scores, so only run it once for each scoring session.

In [ ]:
intensities = pd.read_csv('intens_for_calibration.csv')
samples = pd.read_csv('events_for_calibration.csv')
samples['score'] = None

session_id = ''.join(np.random.choice(list(string.ascii_letters), size=6))

In [ ]:
# replace the session id below with the id you would like to load
session_id = 'xxxxxx'
samples = pd.read_csv('scored_sample_' + session_id + '.csv', index_col=None)

Then we randomize the order that the events are displayed, and allow you to visualize them!

In [ ]:
subsample = samples.loc[~(samples['score'] >= 0), ['cell', 'particle']]

cell_label = widgets.Label(value='')
particle_label = widgets.Label(value='')
progress_string = '(%d/%d)' % (np.sum(samples['score'] >= 0), len(samples))
progress_label = widgets.Label(value=progress_string)
labels = widgets.HBox([cell_label, particle_label, progress_label])

centered_layout = widgets.Layout(align_items = 'center')

loading_widget = widgets.Label(value='Loading...')

puff_butt = widgets.Button(description='Puff!',
                           disabled=False,
                           button_style='',
                           tooltips='Puff',
                           icons='check')
np_butt = widgets.Button(description='Nonpuff!',
                           disabled=False,
                           button_style='',
                           tooltips='Nonpuff',
                           icons='times')
score_butt = widgets.HBox([np_butt, puff_butt])

header = widgets.VBox([labels, score_butt], layout=centered_layout)
summary_display = widgets.HBox([loading_widget])
full_display = widgets.VBox([header, summary_display], layout=centered_layout)
display(full_display)

def display_and_load():
    global events
    global counter
    
    loading_widget.value = "..."
    summary_display.children = [loading_widget]
    
    counter += 1
    if counter == 11:
        loading_widget.value = "Saving..."
        samples.to_csv('scored_sample_' + session_id + '.csv')
        counter = 0
    
    event = next(events, None)
    if event is None:
        loading_widget.value = "Saving..."
        samples.to_csv('scored_sample_' + session_id + '.csv')
        loading_widget.value = "You're done! Sit back, relax, and enjoy the good times!!"
        return
    
    event_intens = intensities.loc[(intensities['cell'] == event[0]) &
                                   (intensities['particle'] == event[1]), :]
    
    loading_widget.value = ("Loading event with length %d..." % len(event_intens['frame'].unique()))
    particle = event[1]
    particle_label.value = str(particle)
    cell = event[0]
    cell_label.value = cell
    progress_string = '(%d/%d)' % (np.sum(samples['score'] >= 0), len(samples))
    progress_label.value = progress_string
    
    summary_widgets = lib.display.plot_summary(event_intens)
    summary_display.children = [widgets.VBox([widgets.HBox(summary_widgets[:2]), 
                                             summary_widgets[2]], layout=centered_layout)]

def call_puff(b):
    samples.loc[(samples['particle'] == int(particle_label.value)) &
                (samples['cell'] == cell_label.value), 'score'] = 1
    display_and_load()

def call_np(b):
    samples.loc[(samples['particle'] == int(particle_label.value)) &
                (samples['cell'] == cell_label.value), 'score'] = 0
    display_and_load()

puff_butt.on_click(call_puff)
np_butt.on_click(call_np)

events = [(cell, particle) for cell, particle in subsample.itertuples(index=False, name=None)]
np.random.shuffle(events)
events = iter((events))

counter = 0
display_and_load()